In [1]:
import pandas as pd
import numpy as np

In [2]:
educacion   = pd.read_csv('~/orga-datos/datos_preprocesados/fiuba_1_postulantes_educacion.csv')

In [10]:
genero_edad = pd.read_csv('~/orga-datos/datos_preprocesados/fiuba_2_postulantes_genero_y_edad.csv')

In [11]:
educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,0,0,0,0,2
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,0,2
4,0zB026d,0,0,0,0,0,3,0


In [4]:
educacion['nombre'].value_counts()

Secundario           185946
Universitario        149734
Terciario/Técnico     66226
Otro                  31775
Posgrado              10256
Master                 3706
Doctorado               266
Name: nombre, dtype: int64

In [5]:
nivel_educativo = pd.get_dummies(educacion['nombre'])
educacion = pd.merge(nivel_educativo, educacion, left_index=True, right_index=True, how='right')

In [6]:
educacion.head()

,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,idpostulante,nombre,estado
0,0,0,0,0,0,0,1,1d2B,Universitario,En Curso
1,0,0,0,0,0,0,1,6M3jr,Universitario,En Curso
2,0,0,0,0,0,1,0,NrlQx,Terciario/Técnico,En Curso
3,0,0,0,0,0,0,1,Y9RKV,Universitario,En Curso
4,0,0,0,0,0,0,1,ZL608,Universitario,En Curso


In [7]:
def set_estado(reg):
    multiplicador = {'En Curso': 1, 'Abandonado': 2, 'Graduado': 3}[reg['estado']]
    reg[reg['nombre']] = reg[reg['nombre']] * multiplicador
    return reg

educacion = educacion.apply(set_estado, axis=1)
educacion = educacion.drop(axis=1, labels=['nombre', 'estado'])
educacion = educacion.groupby('idpostulante').agg(max)

In [8]:
educacion = educacion.reset_index()
educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,0,0,0,0,2
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,0,2
4,0zB026d,0,0,0,0,0,3,0


In [12]:
genero_edad = genero_edad[(genero_edad['sexo'] != 'NO_DECLARA') |  (genero_edad['fechanacimiento'].isna() == False)]

In [13]:
genero_edad[genero_edad.duplicated(subset='idpostulante', keep=False)].sort_values(by='idpostulante').head()

,idpostulante,fechanacimiento,sexo


In [14]:
genero_edad = genero_edad.drop_duplicates(subset='idpostulante', keep='last')

In [15]:
postulantes = pd.merge(educacion, genero_edad, on='idpostulante', how='outer')

In [16]:
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0z5Dmrd,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1965-10-22,MASC
1,0z5JW1r,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1971-04-09,MASC
2,0z5VvGv,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1987-10-18,MASC
3,0zB01pE,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1989-12-01,MASC
4,0zB026d,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1978-02-23,FEM


In [17]:
print('Educación:', len(educacion))
print('Género y edad:', len(genero_edad))
print('Total:', len(postulantes))

Educación: 447909
Género y edad: 480163
Total: 487912


In [18]:
for columna in ['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario', 'Terciario/Técnico', 'Universitario']:
    postulantes[postulantes[columna].isna()] = 0
    postulantes[columna] = postulantes[columna].astype('int')
    postulantes[columna] = postulantes[columna].map(lambda x: {1: 'En Curso', 3: 'Graduado', 2: 'Abandonado', 0:0}[x])

In [19]:
postulantes.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,fechanacimiento,sexo
0,0z5Dmrd,0,0,0,0,0,0,Graduado,1965-10-22,MASC
1,0z5JW1r,0,0,0,0,0,0,Abandonado,1971-04-09,MASC
2,0z5VvGv,0,0,0,0,Graduado,0,0,1987-10-18,MASC
3,0zB01pE,0,0,0,0,0,0,Abandonado,1989-12-01,MASC
4,0zB026d,0,0,0,0,0,Graduado,0,1978-02-23,FEM


In [25]:
postulantes = postulantes.drop_duplicates()

In [26]:
postulantes.to_csv('~/orga-datos/datos_preprocesados/postulantes.csv', index=False)